In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from pathlib import Path
from CGCNN_MT.module.module_utils import calculate_lsv_from_tree, calculate_lse_from_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import BallTree
import pickle

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
log_dir = "./CGCNN_MT/evaluation/TSD_SSD_WS24_water_WS24_water4_WS24_acid_WS24_base_WS24_boiling_seed42_att_cgcnn@version_43"
log_dir = Path(log_dir)
latent_feas = {}
tasks = ["TSD", "SSD", "WS24_water", "WS24_water4", "WS24_acid", "WS24_base", "WS24_boiling"]
labels = {}
for task in tasks:
    for split in [
        'train', 
        'val', 
        'test',
        ]:
        latent_fea = np.load(log_dir / f"{split}_last_layer_fea_{task}.npz")
        latent_feas[f"{task}_{split}"] = latent_fea[list(latent_fea.keys())[0]]
        res_df = pd.read_csv(log_dir / f"{split}_results_{task}.csv")
        labels[f"{task}_{split}"] = res_df['GroundTruth'].values

In [5]:
uncertainty_trees = {}
k = 5
for task in tasks:
    latent_vec_train = latent_feas[f"{task}_train"]
    latent_vec_tree = BallTree(latent_vec_train, metric='minkowski', p=2)  ## metric='euclidean'
    dist_train, ind_train = latent_vec_tree.query(latent_vec_train, k=k, dualtree=True)
    avg_dist_traintrian = dist_train.mean(axis=None)
    tree_dic = {'tree': latent_vec_tree, 
                'avg_dist_traintrian': avg_dist_traintrian, 
                'labels_train': labels[f"{task}_train"],
                'k': k
                }
    if task == "TSD":
        uncertaintys = calculate_lsv_from_tree(tree_dic, latent_feas[f"{task}_test"], k=tree_dic["k"])
    else:
        uncertaintys = calculate_lse_from_tree(tree_dic, latent_feas[f"{task}_test"], k=tree_dic["k"])
    scaler = MinMaxScaler()
    scaler.fit(uncertaintys.reshape(-1, 1))
    tree_dic.update({"scaler": scaler})
    uncertainty_trees[task] = tree_dic

In [6]:
with open(log_dir/"uncertainty_trees.pkl", "wb") as f:
    pickle.dump(uncertainty_trees, f)